In [0]:
# %tensorflow_version 2.1.x
# import tensorflow as tf
# print(tf.__version__)

style_img_path='styles/wave.jpg'

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:

# coding: utf-8
import tensorflow as tf # require tensorflow v2
import tensorflow_addons as tfa
from tensorflow import keras

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
import numpy as np
import os

img_size = 256

# os.environ["CUDA_VISIBLE_DEVICES"] = ""

 

In [0]:

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

In [0]:
print(device_lib.list_local_devices())

In [0]:
# # !mkdir /root/.ssh
# !cp -r "drive/My Drive/.ssh" /root/
# !chmod 600 /root/.ssh/id_rsa
# !chmod 644 /root/.ssh/id_rsa.pub

In [0]:
import os
# os.chdir('/content/Neural-Image-Processing/Fast-Neural-Style-Transfer')
os.chdir('/content/drive/My Drive/github/Neural-Image-Processing/Fast-Neural-Style-Transfer') #
# os.chdir('/home/xing/文档/Git_Repository/Neural-Image-Processing/Fast-Neural-Style-Transfer/')
#print(os.getcwd())

---
## 2.build the transfer model

In [0]:

MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3)) # 通道颜色均值
#k_initializer = tf.truncated_normal_initializer(0, 0.1)
k_initializer = keras.initializers.TruncatedNormal(0, 0.1)
variation_weight = 0.01
kernel_regularizer =  keras.regularizers.l2(variation_weight)
bias_regularizer = keras.regularizers.l2(variation_weight) 
activity_regularizer = keras.regularizers.l2(0.01)
def relu(X):
    return keras.layers.Activation('relu')(X)

def instance_norm(X):
    # return keras.layers.BatchNormalization()(X)
    return tfa.layers.InstanceNormalization(#axis=3, 
                                    center=True, 
                                    scale=True,
                                    beta_initializer="random_uniform",
                                    gamma_initializer="random_uniform") (X)
    #return keras.layers.LayerNormalization()(X)
# kernel_regularizer=kernel_regularizer, bias_regularizer=bias_regularizer ,
# kernel_regularizer=kernel_regularizer, bias_regularizer=bias_regularizer,  
def conv2d(inputs, filters, kernel_size, strides, name = "noname"):
    return keras.layers.Conv2D(filters, kernel_size, strides, padding="same", kernel_initializer=k_initializer, name = name)(inputs)

def deconv2d(inputs, filters, kernel_size = 3, strides = 1, name="noname"):
    #return tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides,  padding="same", kernel_initializer=k_initializer, activity_regularizer=activity_regularizer, name=name)(inputs)
    # shape = tf.shape(inputs)
    shape = inputs.get_shape().as_list()
    
    height, width = shape[1], shape[2]
    # 近邻插值法，
    print(inputs)
    h0 = tf.image.resize(inputs, [height * strides * 2, width * strides * 2], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    return conv2d(h0, filters, kernel_size, strides, name)

    # 残差网络
def residual(inputs, filters = 128, kernel_size = 3, name="noname"):
    X = relu(conv2d(inputs, filters, kernel_size, 1, name=name+"_1"))
    X = conv2d(X, filters, kernel_size, 1, name=name+"_2")
    return keras.layers.Add()([inputs, X])

def get_transfer_model(input_shape=(256, 256, 3), name="style_transfer_net"):
    img_inputs = keras.Input(input_shape, name="inputs")
    #X = tf.pad(img_inputs - MEAN_VALUES, [[0, 0], [10, 10], [10, 10], [0, 0]], mode='reflect')
    X = keras.layers.Subtract()([img_inputs, MEAN_VALUES])
    X = relu(instance_norm(conv2d(X, 32, 9, 1, name="conv1")))
    X = relu(instance_norm(conv2d(X, 64, 3, 2, name="conv2")))
    X = relu(instance_norm(conv2d(X, 128, 3, 2, name="conv3")))

    for i in range(5):
        X = residual(X, 128, 3, name="res"+str(i))

    X = relu(instance_norm(deconv2d(X, 64, 3, 2, name="conv4")))
    X = relu(instance_norm(deconv2d(X, 32, 3, 2, name="conv5")))
    X = keras.layers.Activation('tanh')(instance_norm(conv2d(X, 3, 9, 1, name="conv6")))
    #X = tf.nn.tanh(instance_norm(conv2d(X, 3, 9, 1, name="conv6")))
    X = (X + 1) * (255.0/2)
    #X = keras.layers.Lambda(lambda x: (x+1)*(255.0/2), name="transfer_outputs")(X)
    return keras.Model(inputs=img_inputs, outputs=X, name=name)

In [0]:
# # test transfer model
# test_module = get_transfer_model()
# test_module.summary()

In [0]:
# keras.utils.plot_model(test_model , show_shapes=True)

---
##  import VGG19
### Content feature and Content loss

In [0]:
# content loss
def get_content_loss(content_features, generate_features):
    content_loss = 2 * tf.nn.l2_loss(content_features[2]-generate_features[2]) / tf.cast(tf.size(content_features[2]), dtype=tf.float32)
    return content_loss


In [0]:
# style loss

def get_style_gram(style_features): # input: list of tensor
    grams = []
    for feat in style_features:
        feat = np.reshape(feat, (-1, feat.shape[3]))
        gram = np.matmul(feat.T, feat) / feat.size
        grams.append(gram)
    return grams

def get_style_loss( generate_features, style_features):
    style_grams = get_style_gram(style_features)
    for i in range(len(generate_features)):
        layer = generate_features[i]
        shape = layer.get_shape().as_list()
        bs, height, width, channel = shape[0], shape[1], shape[2], shape[3]
        features = tf.reshape(layer, (-1, height * width, channel))
        gram = tf.matmul(tf.transpose(features, (0,2,1)), features) / (height * width * channel*1.0)
        size = tf.cast(tf.size(layer), tf.float32)
        style_loss = 2 * tf.nn.l2_loss(gram - style_grams[i]) / size
    style_loss = tf.reduce_sum(style_loss, name = 'style_loss')
    return style_loss



## 

In [0]:
# get feature extract model
OUTPUT_LAYERS=["output_feature_" + str(i) for i in range(4)]
STYLE_LAYERS = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']
def get_feature_extract_model(vgg19):
    features_list  = [vgg19.get_layer(layer_name).output for layer_name in STYLE_LAYERS]
    return keras.Model(inputs=vgg19.input, outputs=features_list, name='output_feature')

def get_features(img_path, model):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model(x)



In [0]:
# variation loss
def get_total_variation_loss(inputs):
    h = inputs[:, :-1, :, :] - inputs[:, 1:, :, :]
    w = inputs[:, :, 1:, :]
    h_size = tf.cast(tf.size(h), tf.float32)
    w_size = tf.cast(tf.size(w), tf.float32)
    return tf.nn.l2_loss(h)/ h_size + tf.nn.l2_loss(w) / w_size

In [0]:
# tpu
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
# transfer_model.summary()
# transfer_model.input.shape
# transfer_model.output.shape

In [0]:
# Loss function
with tpu_strategy.scope():
    # with strategy.scope():
    vgg19_1 = VGG19(weights='imagenet', include_top=False)
    # set vgg19 to untrainable
    for layer in vgg19_1.layers:
        layer.trainable = False
    feat_extract_model = get_feature_extract_model(vgg19_1)
    style_features = get_features(style_img_path, feat_extract_model)
    transfer_model = get_transfer_model()

    content_weight = 1
    style_weight = 250
    variation_weight = 0.01

    # x = keras.layers.concatenate([transfer_model.input, transfer_model.output], axis=0)
    # print("666")
    # print(x)
    # y = feat_extract_model(x)
    # content_features = y[0]
    # generate_features = y[1]

    content_features = feat_extract_model(transfer_model.input)
    generate_features = feat_extract_model(transfer_model.output)
    # style_features
    content_loss = get_content_loss(content_features, generate_features)
    style_loss = get_style_loss(generate_features, style_features)
    variation_loss = get_total_variation_loss(transfer_model.output)
    total_loss = content_weight*content_loss + style_weight*style_loss + variation_weight*variation_loss

    transfer_model.add_loss(total_loss)


In [0]:
opt = keras.optimizers.Adam(learning_rate=0.001)
batch_size = 1 * tpu_strategy.num_replicas_in_sync
print(batch_size)
# strategy = tf.distribute.

    # with tpu_strategy.scope():
# with strategy.scope():

def loss_fn(true_val, pred_val):
    return tf.reduce_mean(pred_val) + tf.reduce_mean(true_val)

with tpu_strategy.scope():
    transfer_model.compile(optimizer=opt)

In [0]:
# ## 加载图片
X_data = np.load('train/train2014_5000.preprocessing.npy')
print(X_data.shape)

# X_data = X_data[0:2048, :, :, :]
X_data = X_data[0:4096, :, :, :]
X_data = X_data.astype(np.float32)




In [0]:
# del Y_data
# X_data = X_data.astype(int)
# transfer_model.summary()
# keras.utils.plot_model(transfer_model , show_shapes=True)

In [0]:
# AUTO = tf.data.experimental.AUTOTUNE
# Z = (tf.constant(X_data), tf.constant(X_data))
# data_set = tf.data.Dataset.from_tensor_slices(Z).batch(batch_size).prefetch(AUTO)
# print(data_set)

In [0]:
# # begin training

steps_per_epoch=1024 // batch_size
# transfer_model.fit(X_data, steps_per_epoch=steps_per_epoch, epochs=1)
%time transfer_model.fit(X_data[0:2048, :, :, :], epochs=1)
%time transfer_model.fit(X_data[2048:4096, :, :, :], epochs=1)


In [0]:
# test

sample_img_path= 'content/0.jpg'
sample_size = 256
# , target_size=(sample_size, sample_size)
img = image.load_img(sample_img_path , target_size=(sample_size, sample_size))
x = image.img_to_array(img)
plt.imshow(x.astype(int))
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)





In [0]:
result = transfer_model(x)
plt.axis('off')
# print(result.numpy()[0].astype(int))
plt.imshow(result.numpy()[0].astype(int))


In [0]:
# keras.models.save_model(
#     transfer_model,
#     'models/111.h5'
# )
transfer_model.save('models/222.h5')


In [0]:
# transfer_model_v2 = keras.models.load_model('models/111.tf')
md = keras.models.load_model('models/222.h5',  {'loss_fn': loss_fn})

In [0]:
result = md.predict(x)
plt.axis('off')
plt.imshow(result[0].astype(int))

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
!python '/content/drive/My Drive/github/fast-neural-style-tensorflow/train.py' -c conf/lu.yml #

# python36 train.py -c conf/lu.yml